# VM Environment Setup (No Need to Run Every Time)




**install spacy**




In [ ]:
# 
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_trf
!pip install scispacy
!pip install "scispacy_model/en_core_sci_scibert-0.5.1.tar.gz"
#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 167, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 247, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 370, in run
    reqs, check_supported_wheels=not options.target_dir
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 93, in resolve
    collected.requirements, max_rounds=try_to_avoid_resolution_too_deep
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 481, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 348, in resolve
    self._add_to_criteria(self.state.criteria, r, paren

In [ ]:
!python -m spacy validate

2022-10-03 00:29:11.312643: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.4.1) =================
ℹ spaCy installation: /usr/local/lib/python3.7/dist-packages/spacy

NAME                  SPACY            VERSION                            
en_ner_bc5cdr_md      >=3.0.1,<3.1.0   0.4.0   --> n/a       
en_core_sci_sm        >=3.4.1,<3.5.0   0.5.1   ✔
en_core_sci_scibert   >=3.4.1,<3.5.0   0.5.1   ✔
en_core_web_trf       >=3.4.0,<3.5.0   3.4.0   ✔

ℹ The following packages are custom spaCy pipelines or not available
for spaCy v3.4.1:
en_ner_bc5cdr_md


In [ ]:
!pip install torch==1.8.0

**install OpenNRE**

In [ ]:
!git clone https://github.com/thunlp/OpenNRE.git

In [ ]:
%cd OpenNRE

In [ ]:
pip install -r requirements.txt

In [ ]:
!python setup.py install

In [ ]:
%cd ../

**install sentencepiece for transformers dependencies**

In [ ]:
!pip install transformers==4.11.0 # last version 4.1.0
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 69.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.1.0
    Uninstalling transformers-4.1.0:
      Successfully uninstalled transformers-4.1.0


**install pubmed-parser**

In [ ]:
!pip install pubmed-parser

**install gensim**

In [ ]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 5.2 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


# Main Scripts

**Connect to MongoDB and Load PubMed Data**

In [ ]:
import pymongo
client = pymongo.MongoClient("mongodb://cosmosdbcolumbiacapstonefall2022:alv0mZxNDjmu37OvR0USNaFrT694JwXE8qpdmb62KSRhHZbZIHaiIw7KqceRlSwFRmhhqFCfdtCKDEnWXh9w8Q==@cosmosdbcolumbiacapstonefall2022.mongo.cosmos.azure.com:10255/?ssl=true&replicaSet=globaldb&retrywrites=false&maxIdleTimeMS=120000&appName=@cosmosdbcolumbiacapstonefall2022@")
# Database Name
db = client['pubmed']
# Collection Name
col = db['abstracts']

abstracts = col.find({},{'_id':0, 'text':1})
abstracts = [a['text'] for a in abstracts]

In [ ]:
print(abstracts[0])

Severe COVID-19 is associated with venous thromboembolic events and and immuno-thrombotic phenomena, responsible for pulmonary vascular damage. This review summarizes the current knowledge on thrombotic risk in COVID-19 inpatients, the potential predictive factors (including D-dimer) and the randomized trials studying the effect of intermediate or therapeutic-dose anticoagulation on the clinical and thrombotic prognosis. Despite the initial hope, therapeutic anticoagulation does not improve the clinical prognosis in critically ill inpatients, and standard prophylactic anticoagulation is therefore recommended. In non-critical inpatients, the use of therapeutic anticoagulation may help reduce the risk of severe clinical deterioration, but its risk-benefit will be clarified in ongoing studies and meta-analyzes.


**Use NER to extract entities**

In [ ]:
import pickle
import spacy
%cd /content
with open("/content/simple_dict.txt","rb") as file:
  simple_dict = pickle.load(file)
nlp = spacy.load("en_core_sci_scibert")

def entity_extraction(text_input):
  doc = nlp(text_input)
  text_entity_list = []
  for sentence in doc.sents:
    sentence_entity_list = []
    entity_start = -1
    entity_end = -1
    and_keyword_connecting = False
    prev_adp = False

    for token in sentence:
      if (token.ent_type != 0 and token.tag_ == 'JJ'):
        if entity_start == -1: # Entering a new entity
          entity_start = token.idx
      elif (token.ent_type != 0 and token.pos_ == 'NOUN'):
        if entity_start == -1: # Entering a new entity
          entity_start = token.idx
        entity_end = token.idx + len(token) # Possible to end an entity here
      elif token.ent_type != 0 and not prev_adp:
        entity_start = -1
        entity_end = -1
        prev_adp = False
      elif (token.text == "of"): #ignore the previous entity
        entity_start = -1
        entity_end = -1
      elif (token.pos_ == 'ADP'):
        prev_adp = True
      elif (token.text == "and" and entity_end == -1): #if not yet reaches the end of an entity
        continue
      else: 
        #checks if an entity has been formed
        if (entity_start != -1 and entity_end != -1):
          if and_keyword_connecting and (entity_end - sentence_entity_list[-1]['end'] <= 30): 
            #if close enough with previous entity and connected by "AND"
            sentence_entity_list[-1]['end'] = entity_end
          else: 
            #if is an independent entity
            sentence_entity_list.append({"start": entity_start, "end": entity_end, "label": "Entity"})
          and_keyword_connecting = False

        elif token.pos_ == "NOUN" and and_keyword_connecting:
          #if is a noun (but not recognized as an entity) and should be connected by AND keyword
          if (token.idx + len(token) - sentence_entity_list[-1]['end'] <= 30):
            sentence_entity_list[-1]['end'] = token.idx + len(token)
          and_keyword_connecting = False
        
        if token.text == 'and':
          and_keyword_connecting = True

        entity_start = -1
        entity_end = -1


    for ent in sentence_entity_list:
      start_index = ent['start']
      end_index = ent['end']
      entity_text = doc.text[start_index:end_index].lower()

      #Only consider it to be an entity if it is not a simple word
      if entity_text not in simple_dict:
        text_entity_list.append(ent)



  ex = {
      "text": doc.text,
      "ents": text_entity_list,
      "title": None
  }

  spacy.displacy.render(ex , style='ent', manual=True, jupyter=True)

  return ex

/content


/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [ ]:
input_text = abstracts[10]
ex = entity_extraction(input_text)

/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/autocast_mode.py:117: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")


**auto-summarization transformers**

CNN XSUM

In [34]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

# Download configuration from huggingface.co (user-uploaded) and cached, use cache_dir to change cache dir
tokenizer = AutoTokenizer.from_pretrained("sysresearch101/t5-large-finetuned-xsum-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("sysresearch101/t5-large-finetuned-xsum-cnn")

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [35]:
# abstract[10][0]
import torch 
ARTICLE_TO_SUMMARIZE = '''Extracellular matrixes (ECMs), such as the cell walls and biofilms,
are important for supporting cell integrity and function and regulating intercellular communication.'''
ARTICLE_TO_SUMMARIZE_MASKED = '''[MASK], such as the [MASK], 
are important for supporting [MASK] and regulating [MASK].'''

# code to mask entity
'''
entity = ["Extracellular", 'matrixes', '(ECMs)', 'cell']
split_words = ARTICLE_TO_SUMMARIZE.split(' ')
masked_index = []
for i, word in enumerate(split_words):
  if word in entity:
    masked_index.append(i) 

match_idxs = torch.LongTensor([[wid not in masked_index for wid in encoded.word_ids(0)]])
print("Subword indices of matching word", match_idxs)

# Merge: if a word is zero in our custom match, merge, if not, use the original mask
# This ensures that we mask the word IDs but keep the original mask for special tokens (cls, pad, etc.)
merged = torch.where(match_idxs == 0, match_idxs, encoded["attention_mask"])
print("Merged mask", merged)
'''

'\nentity = ["Extracellular", \'matrixes\', \'(ECMs)\', \'cell\']\nsplit_words = ARTICLE_TO_SUMMARIZE.split(\' \')\nmasked_index = []\nfor i, word in enumerate(split_words):\n  if word in entity:\n    masked_index.append(i) \n\nmatch_idxs = torch.LongTensor([[wid not in masked_index for wid in encoded.word_ids(0)]])\nprint("Subword indices of matching word", match_idxs)\n\n# Merge: if a word is zero in our custom match, merge, if not, use the original mask\n# This ensures that we mask the word IDs but keep the original mask for special tokens (cls, pad, etc.)\nmerged = torch.where(match_idxs == 0, match_idxs, encoded["attention_mask"])\nprint("Merged mask", merged)\n'

In [36]:
input_ids = tokenizer(ARTICLE_TO_SUMMARIZE_MASKED, return_tensors='pt')
input_ids

{'input_ids': tensor([[  784, 23010,   439, 13679,   224,    38,     8,   784, 23010,   439,
         13679,    33,   359,    21,  3956,   784, 23010,   439,   908,    11,
             3, 27321,   784, 23010,   439,  4275,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]])}

problem:
1. 1 to many relationship. i.e. ECMs support cell integrity and function; ECMs regulate intercellular communication. How to identify this and use ECMs twice
2. mask ner and decode

In [37]:
input_ids = tokenizer.encode(ARTICLE_TO_SUMMARIZE_MASKED, return_tensors='pt')
summary_ids = model.generate(input_ids,
            min_length=20,
            max_length=80,
            num_beams=10,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.8,
            top_k = 50,
            top_p = 0.95)

summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

[MASK] is a global network of people, organisations and institutions that work together to improve the lives of children.


In [38]:
after_ner = '''A are important for supporting B'''

In [39]:
input_ids = tokenizer.encode(after_ner, return_tensors='pt')
summary_ids = model.generate(input_ids,
            min_length = 1,
            max_length=80,
            num_beams=10,
            repetition_penalty=2.5,
            length_penalty=5.0,
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.8,
            top_k = 50,
            top_p = 0.95)

summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)

A and B are important for supporting the development of C.


In [ ]:
len(input_ids[0])

7

Model not working. Add in non-related info

In [40]:
# explore other models
import re
from transformers import AutoTokenizer, T5ForConditionalGeneration

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

model_name = "JulesBelveze/t5-small-headline-generator"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [41]:

input_ids = tokenizer(
    [WHITESPACE_HANDLER(ARTICLE_TO_SUMMARIZE)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=20
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=20,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)
print(summary)

# change hyperparameter
input_ids = tokenizer(
    [WHITESPACE_HANDLER(ARTICLE_TO_SUMMARIZE)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=80
)["input_ids"]

output_ids = model.generate(input_ids,
            min_length=20,
            max_length=80,
            num_beams=10,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            no_repeat_ngram_size=2,
            use_cache=True,
            do_sample = True,
            temperature = 0.8,
            top_k = 50,
            top_p = 0.95
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)
print(summary)

-Matrix Extracellular Matrixes (ECMs), such
: Extracellular matrixes (ECMs), such as cell walls and biofilms


Truncate the text. 
Most models work with long texts. i.e. summarizing an article to 1 sentence

In [46]:
# scifive: https://github.com/justinphan3110/SciFive

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("razent/SciFive-large-Pubmed_PMC-MedNLI")  
model = AutoModelForSeq2SeqLM.from_pretrained("razent/SciFive-large-Pubmed_PMC-MedNLI")

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [49]:
sent_1 = "Extracellular matrixes (ECMs) are important for supporting cell integrity and function and regulating intercellular communication"
sent_2 = "cell integrity and function and regulating intercellular communication"
text =  f"mednli: sentence1: {sent_1} sentence2: {sent_2}"

encoding = tokenizer.encode_plus(sent_1, padding='max_length', max_length=50, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"], encoding["attention_mask"]
outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    max_length=30,
    length_penalty = 10,
    early_stopping=False
)

for output in outputs:
    line = tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print(line)

.. (ICs) are a major component of the extracellular matrix (ECM) that are deposited by cells


In [ ]:
text

'mednli: sentence1: In the ED, initial VS revealed T 98.9, HR 73, BP 121/90, RR 15, O2 sat 98% on RA. sentence2: The patient is hemodynamically stable'

# Even Worse Model

In [28]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, PegasusForConditionalGeneration, PegasusTokenizer

In [29]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

#article_text = abstracts[0]

In [42]:
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [43]:
input_ids = tokenizer(
    [WHITESPACE_HANDLER(ARTICLE_TO_SUMMARIZE)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=10
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=10,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

In our series of letters from scientist


In [44]:
model2_name = "google/pegasus-xsum"
tokenizer2 = PegasusTokenizer.from_pretrained(model2_name)
model2 = PegasusForConditionalGeneration.from_pretrained(model2_name)

/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [45]:
batch = tokenizer2(ARTICLE_TO_SUMMARIZE, truncation=True, padding="longest", return_tensors="pt")
translated = model2.generate(**batch)
tgt_text = tokenizer2.batch_decode(translated, skip_special_tokens=True)
tgt_text

['The aim of this study is to understand the role of extracellular matrixes in the development and maintenance of healthy human tissues.']